**_[Hacker News](http://news.ycombinator.com/)_** is a community where users can submit articles, and other users can upvote those articles. The articles with the most upvotes make it to the front page, where they're more visible to the community.

Our dataset consists of submissions users made to Hacker News from 2006 to 2015. Moreover, we have sampled `3000` rows randomly from [here](https://github.com/arnauddri/hn) and we removed all of the extraneous columns. Our data only has four columns:

- `submission_time` - When the article was submitted
- `upvotes` - The number of upvotes the article received
- `url` - The base URL of the article
- `headline` - The article's headline

In this mission, we'll be predicting the number of upvotes the articles received, based on their headlines. Because upvotes are an indicator of popularity, we'll discover which types of articles tend to be the most popular.

In [1]:
import pandas as pd
submissions = pd.read_csv("sel_hn_stories.csv")
submissions.columns = ["submission_time", "upvotes", "url", "headline"]
submissions = submissions.dropna()

Our goal is to train a linear regression algorithm that predicts the number of upvotes a headline would receive. To do this, we'll need to convert each headline to a numerical representation.

While there are several ways to accomplish this, we'll use a bag of words model. A `bag of words model` represents each piece of text as a numerical vector.

We'll examine each step in the bag of words process in this mission. For now, here's a high-level diagram showing how two sentences, `I rode my horse to Berlin.` and `You rode my horse to Berlin in the winter.`, convert to a bag of words:

<img src='image_1.jpg'>

The first step in creating a bag of words model is `tokenization`. In tokenization, we break a sentence up into disconnected words.

Here's a diagram in which we tokenize the two sentences we mentioned above:

<img src='image_2.jpg'>

As you can see, all we're doing is splitting each sentence into a list of individual words, or tokens. The split occurs on the space character (`" "`).

In [2]:
tokenized_headlines = []

for word in submissions.headline:
    tokenized_headlines.append(word.split())

We now have tokens, but we need to process them a bit to make our predictions more accurate. We know that `Berlin`, `Berlin.`, and `berlin` all refer to the same word, but the computer doesn't know that. We'll need to convert those variations so that they're consistent.

We can do this by lowercasing (which will convert `Berlin` to `berlin`), and also by removing punctuation (so `Berlin.` becomes `Berlin`).

<img src='image_3.jpg'>

Preprocessing doesn't have to be perfect, but the more we can help the computer group the same word together, the higher our prediction accuracy will be. Take a look through your tokens, and see if there are any instances of the same word that you haven't grouped together.

In [3]:
punctuation = [",", ":", ";", ".", "'", '"', "’", "?", "/", "-", "+", "&", "(", ")"]
clean_tokenized = []

for token_list in tokenized_headlines:
    tokens = []
    for token in token_list:
        token = token.lower()
        for punc in punctuation:
            token = token.replace(punc,'')
        tokens.append(token)
    clean_tokenized.append(tokens)

Now that we have our tokens, we can begin converting the sentences to their numerical representations. First, we'll retrieve all of the unique words from all of the headlines. Then, we'll create a matrix, and assign those words as the column headers. We'll initialize all of the values in the matrix to `0`.

<img src='image_4.jpg'>

We'll use a pandas dataframe instead of a NumPy matrix. We can create a dataframe with all zero values using this syntax:

> pd.DataFrame(0, index=np.arange(len(clean_tokenized)), columns=unique_tokens)

The code above will create a dataframe with as many rows as the number of items in `clean_tokenized`. Each column name will be a word from `unique_tokens`. This assumes that we already assigned the unique tokens to `unique_tokens`. Each cell in the dataframe will have the value `0`.

In [4]:
import numpy as np
unique_tokens = []
single_tokens = []

for lists_of_token in clean_tokenized:
    for token in lists_of_token:
        if token not in single_tokens:
            single_tokens.append(token)
        elif token in single_tokens and token not in unique_tokens:
            unique_tokens.append(token)
    
counts = pd.DataFrame(0, index=np.arange(len(clean_tokenized)), columns=unique_tokens)

Now that we have a matrix where all values are `0`, we need to fill in the correct counts for each cell. This involves going through each set of tokens, and incrementing the column counters in the appropriate row.

<img src='image_5.jpg'>

When we're finished, we'll have a row vector for each headline that tells us how many times each token occured in that headline.

To accomplish this, we can loop through each list of tokens in `clean_tokenized`, then loop through each token in the list and increment the proper cell.

In [5]:
for index, tokens_list in enumerate(clean_tokenized):
    for token in tokens_list:
        if token in unique_tokens:
            counts.iloc[index][token] += 1

We have over `2000` columns in our matrix. This can make it very hard for a linear regression model to make good predictions. Too many columns will cause the model to fit to noise instead of the signal in the data.

There are two kinds of features that will reduce prediction accuracy. Features that occur only a few times will cause overfitting, because the model doesn't have enough information to accurately decide whether they're important. These features will probably correlate differently with upvotes in the test set and the training set.

Features that occur too many times can also cause issues. These are words like `and` and `to`, which occur in nearly every headline. These words don't add any information, because they don't necessarily correlate with upvotes. These types of words are sometimes called stopwords.

To reduce the number of features and enable the linear regression model to make better predictions, we'll remove any words that occur fewer than 5 times or more than `100` times.

In [6]:
word_counts = counts.sum(axis=0)

counts = counts.loc[:, (word_counts >= 5) & (word_counts <= 100)] 

Now we'll need to split the data into two sets so that we can evaluate our algorithm effectively. We'll train our algorithm on a training set, then test its performance on a test set.

The `train_test_split` function from scikit-learn will help us accomplish this.

We'll pass in `.2` for the `test_size` parameter to randomly select `20%` of the rows for our test set, and `80%` for our training set.

`X_train` and `X_test` contain the predictors, and `y_train` and `y_test` contain the value we're trying to predict (upvotes).

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(counts, submissions["upvotes"], test_size=0.2, random_state=1)

Now that we have a training set and a test set, let's train a model and make test predictions. We'll use a linear regression algorithm from scikit-learn.

First we'll initialize the model using the `LinearRegression` class. Then, we'll use the `fit()` method on the model to train with `X_train` and `y_train`. Finally, we'll make predictions with `X_test`.

When we make predictions with a linear regression model, the model assigns coefficients to each column. Essentially, the model is determining which words correlate with more upvotes, and which with less. By finding these correlations, the model will be able to predict which headlines will be highly upvoted in the future. While the algorithm won't have a high level of understanding of the text, linear regression can generate surprisingly good results.

In [8]:
from sklearn.linear_model import LinearRegression

clf = LinearRegression()

clf.fit(X_train, y_train)

predictions = clf.predict(X_test)

Now that we have predictions, we can calculate our prediction error. We'll need to select an error metric first, though. We'll use `mean squared error` (MSE), which is a common error metric.

Here's the formula for MSE:

$ MSE = \frac{1}{n}\sum_{i=1}^{n}(\hat{Y_{i}} - Y_{i})^{2} $

With MSE, we subtract the predictions from the actual values, square the results, and find the mean. Because the errors are squared, MSE penalizes errors further away from the actual value more than those close to the actual value. We want to use MSE because we'd like all of our predictions to be relatively close to the actual values.

In [9]:
mse = ((predictions - y_test) ** 2).sum()/ len(predictions)

In [10]:
mse

2651.1457056689637

Our MSE is `2651`, which is a fairly large value. There's no hard and fast rule about what a "good" error rate is, because it depends on the problem we're solving and our error tolerance.

In this case, the mean number of upvotes is `10`, and the standard deviation is `39.5`. If we take the square root of our MSE to calculate error in terms of upvotes, we get `51.5`. This means that our average error is `51.5` upvotes away from the true value. This is higher than the standard deviation, so our predictions are often far off-base.

We can take several steps to reduce the error and explore natural language processing further. Here are some ideas for your next steps:

- Use the entire data set. While we used samples in this mission, we could download the entire data set from [here](https://github.com/arnauddri/hn). This approach will reduce the error rate dramatically. There are many features in natural language processing. Using more data will ensure that the model will find more occurrences of the same features in the test and training sets, which will help the model make better predictions.
- Add "meta" features like headline length and average word length.
- Use a random forest, or another more powerful machine learning technique.
- Explore different thresholds for removing extraneous columns.